In [36]:
import requests
import pandas as pd
import json


In [37]:
API_KEY = "a2faf0838c2aacce44be67cea8b40c06-48d69bac26199a4d5850294881134c34"
ACCOUNT_ID = "101-004-21509763-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [38]:
session = requests.Session()

In [39]:
session.headers.update({
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {API_KEY}'
})

In [40]:
params = dict(
    count = 10,
    granularity = "H1"
)

In [41]:
url = f"{OANDA_URL}/instruments/EUR_USD/candles"

In [42]:
response = session.get(url, params=params, data=None, headers=None)

In [43]:
response.status_code

200

In [44]:
response.json()

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 3016,
   'time': '2023-11-03T11:00:00.000000000Z',
   'mid': {'o': '1.06430', 'h': '1.06528', 'l': '1.06409', 'c': '1.06467'}},
  {'complete': True,
   'volume': 12095,
   'time': '2023-11-03T12:00:00.000000000Z',
   'mid': {'o': '1.06468', 'h': '1.07194', 'l': '1.06455', 'c': '1.07094'}},
  {'complete': True,
   'volume': 12529,
   'time': '2023-11-03T13:00:00.000000000Z',
   'mid': {'o': '1.07095', 'h': '1.07248', 'l': '1.06961', 'c': '1.07172'}},
  {'complete': True,
   'volume': 12965,
   'time': '2023-11-03T14:00:00.000000000Z',
   'mid': {'o': '1.07223', 'h': '1.07359', 'l': '1.06936', 'c': '1.07199'}},
  {'complete': True,
   'volume': 7011,
   'time': '2023-11-03T15:00:00.000000000Z',
   'mid': {'o': '1.07200', 'h': '1.07355', 'l': '1.07154', 'c': '1.07324'}},
  {'complete': True,
   'volume': 5952,
   'time': '2023-11-03T16:00:00.000000000Z',
   'mid': {'o': '1.07322', 'h': '1.07394', 